In [2]:
#Importing Libraries
import nltk
from nltk.stem import WordNetLemmatizer
from PIL import Image as pil
import tflearn
import tensorflow as tf
tf.compat.v1.disable_v2_behavior
import numpy as np
import random
import json
import pickle
from flask import Flask, request, jsonify

stemmer = WordNetLemmatizer()

In [3]:
with open('intents.json') as data:
    intents = json.load(data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?','.','!',',']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Tokenizing each word
        w = nltk.word_tokenize(pattern,language="english", preserve_line=False)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]))
classes = sorted(list(set(classes)))


print('Documents: ',len(documents))
print('Classes: ',len(classes),classes)
print(len(words),"Stemmed Words",words)

Documents:  34
Classes:  8 ['goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'substitute', 'thanks', 'today']
61 Stemmed Words ["'s", 'accept', 'alternative', 'any', 'anyone', 'are', 'baking', 'butter', 'bye', 'can', 'card', 'care', 'cash', 'cooky', 'credit', 'curd', 'day', 'do', 'egg', 'flour', 'for', 'good', 'goodbye', 'great', 'greeting', 'have', 'hello', 'helpful', 'hi', 'hour', 'how', 'i', 'in', 'instead', 'is', 'later', 'leaving', 'mastercard', 'morning', "n't", 'now', 'of', 'only', 'open', 'replace', 'see', 'substitute', 'sugar', 'take', 'thank', 'thanks', 'that', 'there', 'to', 'today', 'use', 'what', 'when', 'with', 'you', 'your']


In [5]:
#Creating Training data
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#Shuffling features and turning into array
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = [entry[0] for entry in training]
train_y = [entry[1] for entry in training]

train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

#print(train_x[0])
#print(train_y[0])

In [6]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()

# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y, n_epoch=1000, batch_size=16, show_metric=True)

model.save('model.tflearn')

Training Step: 2999  | total loss: 0.05523
| Adam | epoch: 1000 | loss: 0.05523 - acc: 0.9999 -- iter: 32/34
Training Step: 3000  | total loss: 0.05492
| Adam | epoch: 1000 | loss: 0.05492 - acc: 0.9999 -- iter: 34/34
--
INFO:tensorflow:c:\Users\Diya\OneDrive\Desktop\Programs\Chatbot\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
#Pickling data
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [8]:
#Importing chatbot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)
    
#Unpickling data
data = pickle.load(open('training_data', 'rb'))
words = data['words']
classes = data['classes']

train_x = data['train_x']
train_y = data['train_y']

In [9]:
tf.compat.v1.reset_default_graph()

#Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
try:
    model.load('model.tflearn')
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading the model: {e}")

INFO:tensorflow:Restoring parameters from c:\Users\Diya\OneDrive\Desktop\Programs\Chatbot\Chatbot\model.tflearn
Model loaded successfully!


In [10]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    bow = bag_of_words(sentence, words)
    results = model.predict([bow])[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{'intent': classes[r[0]], 'probability': str(r[1])} for r in results]

def get_response(intents_list, intents_json):
    if not intents_list:
        return "Sorry, I didn't understand that."
    tag = intents_list[0]['intent']
    for intent in intents_json['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])